# Citibike Stations' Inflow/Outflow Ratio
Group Members:
Cindy Y. Liu, Yuxiang Zhang

In [1]:
# Load Default settings
import pylab as pl
import pandas as pd
import numpy as np
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Load Citibike data
df = pd.read_csv('201508-citibike-tripdata.csv')

In [3]:
# Drop unnecessary features
df = df.drop(['tripduration','starttime','stoptime'],axis=1)

In [6]:
# Store the information of stations into a csv file which includes longitude,latitude and station id
start = pd.DataFrame({'ID': df['start station id'], \
                    'Latitude': df['start station latitude'],\
                    'Longitude':df['start station longitude']})
end = pd.DataFrame({'ID': df['end station id'], \
                    'Latitude': df['end station latitude'],\
                    'Longitude':df['end station longitude']})
merged = pd.concat([start,end],axis=0)
merged = merged.drop_duplicates('ID')
merged.to_csv('station.csv')    

We use the Geopy package to [reverse geocoding](https://en.wikipedia.org/wiki/Reverse_geocoding) by converting geographic coordinates (latitude and longitude) into street addresses. Based on these, we get the corresponding borough information for each Citibike station and store the information into a csv file named 'stationbyborough'.

In [7]:
from geopy.geocoders import Nominatim
import numpy as np
stations = pd.read_csv('station.csv')
output = np.empty(len(stations), dtype='object')
for index,station in stations.iterrows():
    geolocator = Nominatim()
    location = geolocator.reverse("%s,%s"%(str(station['Latitude']), str(station['Longitude'])),timeout=10)
    if location.raw["address"]["county"]=="New York County"or location.raw["address"]["county"]=="New York":
        output[index]="Manhattan"
    else:
        if location.raw["address"]["county"]=="Kings County":
            output[index]="Brooklyn"
        else:
            if location.raw["address"]["county"]=="Queens County":
                output[index]="Queens"
            else:
                output[index]=location.raw["address"]["county"]

Then we store the information into a csv file named 'stationbyborough'.

In [8]:
output.tofile('stationbyborough.csv',sep='\n')

In [9]:
# Load stations by borough csv file
stations_by_borough = pd.read_csv('stationbyborough.csv')

In [10]:
# Create empty arrays for storing the number of inflow trips, outflow trips, and the ratio of inflow trips to total trips for each citibike station
in_jour, out_jour, in_out_ratio = np.zeros(len(stations_by_borough)),np.zeros(len(stations_by_borough)),np.zeros(len(stations_by_borough))

In [11]:
# Iterate through all the stations to get the total number of inflow and outflow trips of each station
for index, station in stations_by_borough.iterrows():
    start = sum(df['start station id']==station['id'])
    end = sum(df['end station id']==station['id'])
    in_jour[index]= start
    out_jour[index]= end
    in_out_ratio[index] = start/float(end+start)

KeyError: 'id'

In [ ]:
# Create a new data frame which includes all the inflow and outflow information of each station
results = pd.DataFrame({'sid':stations_by_borough['id'],\
                      'inflow':in_jour,\
                      'outflow':out_jour,\
                      'ratio':in_out_ratio,\
                      'borough':stations_by_borough['BoroName']})
cols = ['sid','inflow','outflow','ratio','borough']
results = results[cols]
results.to_csv('results.csv')

In [ ]:
# Display the summary statistics for the new data frame
results.describe()

In [ ]:
# Calculate the number of both Manhattan and Brooklyn stations
pd.value_counts(results['borough'].values.ravel())

## Statistical Analysis

In [ ]:
# Create new dataframes for Manhattan and Brooklyn seperately
manhattan = results[results['borough']=='Manhattan']
brooklyn = results[results['borough']=='Brooklyn']

norm_m = 1
norm_b = 1

error_m = np.sqrt(manhattan['ratio'].count())
error_b = np.sqrt(brooklyn['ratio'].count())

In [ ]:
# Create histograms for Manhattan and Brooklyn's inflow to outflow trips ratio
fig = pl.figure(figsize(10,10))
gs = gridspec.GridSpec(2, 1)
ax = fig.add_subplot(gs[0,0])
manhattan['ratio'].plot(kind="hist",yerr=[((error_m)/norm_m,(error_m)/norm_m)],\
                        bins=30,color='IndianRed', alpha=0.5,normed=1)
ax.set_title('Manhattan Stations')
bx = fig.add_subplot(gs[1,0])
brooklyn['ratio'].plot(kind="hist",yerr=[((error_b)/norm_b,(error_b)/norm_b)],\
                       bins=30, alpha=0.5,normed=1)
bx.set_title('Brooklyn Stations')

In [ ]:
# Display summary statitics for Manhattan and Brooklyn's inflow to outflow trips ratio
manhattan['ratio'].describe()
brooklyn['ratio'].describe()

# Hypothesis: 
Manhattan's citibike stations have less inflow trips than Brooklyn.

# Z test for the mean of inflow to total trips ratios for two samples

$H_{0} : p_m-p_b>0$

$H_{\alpha}: p_m-p_b<=0$

In [ ]:
# Calculate the mean and standard deviation for two samples
mean_m = np.mean(manhattan['ratio'])
mean_b = np.mean(brooklyn['ratio'])
std_m = np.std(manhattan['ratio'])
std_b = np.std(brooklyn['ratio'])
n_m = len(manhattan)
n_b = len(brooklyn)

# Create formula to calculate the standard error for the difference
SE = lambda sd1,sd2,n1,n2: np.sqrt(sd1**2/float(n1)+sd2**2/float(n2))
se = SE(std_m, std_b, n_m, n_b)

In [ ]:
# Use the two sample z-test to evaluate the difference between two groups
z_1 = (mean_m - mean_b)/se
print z_1

In [ ]:
# According to the Z-score to, compute the p-value and compare against the critical value.
p_1 = 0.5319
alpha = 0.05
def report_result(p,a):
    print 'Is the p value {0:.2f} smaller than the critical value {1:.2f}? '.format(p,a)

    print '{0}, the Null hypothesis is {1}'.format( 'Yes' if p<a  else 'No','rejected'if p<a else'not rejected')

    
report_result(p_1,alpha)

# Z test for the proportion of trips which have more inflow trips than outflow trips (ratio > 0.5) to the total trips  in two samples.

In [ ]:
P_0 = sum(manhattan['ratio']>=0.5)/float(len(manhattan))
P_1 = sum(brooklyn['ratio']>=0.5)/float(len(brooklyn))

n_0 = len(manhattan)
n_1 = len(brooklyn)

Nt_0 = P_0*n_0
Nt_1 = P_1*n_1
print (P_0, P_1, n_0, n_1)

# Pooled Sample Proportion
psp =(P_0*n_0+P_1*n_1)/(n_1+n_0)

SE = lambda p, n1, n2: np.sqrt(psp*(1-psp)*(1/float(n1)+1/float(n2)))

se = SE(P_0,n_0,n_1)

# Z-score
zscore = lambda p0, p1, s : (p0-p1)/s
z_2 = zscore(P_1, P_0, se)
print(z_2)

In [ ]:
p = 1 - 0.6179
alpha = 0.05
def report_result(p,a):
    print 'Is the p value {0:.2f} smaller than the critical value {1:.2f}? '.format(p,a)
   
    print '{0}, the Null hypothesis is {1}'.format( 'Yes' if p<a  else 'No','rejected'if p<a else'not rejected')

report_result(p,alpha)